In [1]:
import numpy as np
import pandas as pd
import time
import copy

### 1. 初始化二维数组

In [2]:
N = []
for i in range(7):
    N.append([0]*7)

N = pd.DataFrame(N)

N.iloc[0, 1] = N.iloc[1, 0] = 0.58
N.iloc[0, 4] = N.iloc[4, 0] = 0.14
N.iloc[0, 6] = N.iloc[6, 0] = 0.94
N.iloc[3, 6] = N.iloc[6, 3] = 0.14
N.iloc[2, 1] = N.iloc[1, 2] = 0.14
N.iloc[5, 1] = N.iloc[1, 5] = 0.14
N.iloc[5, 2] = N.iloc[2, 5] = 0.14 

labels = [0, 1, 0, 0, 1, 0, 0]
N

,0,1,2,3,4,5,6
0,0.00,0.58,0.00,0.00,0.14,0.00,0.94
1,0.58,0.00,0.14,0.00,0.00,0.14,0.00
2,0.00,0.14,0.00,0.00,0.00,0.14,0.00
3,0.00,0.00,0.00,0.00,0.00,0.00,0.14
4,0.14,0.00,0.00,0.00,0.00,0.00,0.00
5,0.00,0.14,0.14,0.00,0.00,0.00,0.00
6,0.94,0.00,0.00,0.14,0.00,0.00,0.00


### 将与每个节点相连的坏节点的权重值加在一起并赋予新值

In [3]:
# 行中坏的列进行相加
# for i in range(7):
#     sum_bad = np.dot(N[i], labels)
#     for j in range(7):
#         if N.iloc[i, j]>0:
#             N.iloc[i, j] = (1-labels[j])* N.iloc[i, j] + labels[j]*sum_bad

# 列中坏的行进行相加
# for i in range(7):
#     sum_bad = np.dot(N[i], labels)
#     for j in range(7):
#         if N.iloc[j, i]>0:
#             N.iloc[j, i] = (1-labels[j])* N.iloc[j, i] + labels[j]*sum_bad

N.iloc[0, 4 ] = 0.58

N

,0,1,2,3,4,5,6
0,0.00,0.58,0.00,0.00,0.58,0.00,0.94
1,0.58,0.00,0.14,0.00,0.00,0.14,0.00
2,0.00,0.14,0.00,0.00,0.00,0.14,0.00
3,0.00,0.00,0.00,0.00,0.00,0.00,0.14
4,0.14,0.00,0.00,0.00,0.00,0.00,0.00
5,0.00,0.14,0.14,0.00,0.00,0.00,0.00
6,0.94,0.00,0.00,0.14,0.00,0.00,0.00


### 3. 求图的转移概率矩阵
$p_i$点的转移概率为：
$\frac{w_{ij}}{degree(p_i)}$，

其中$degree(p_i)$为$p_i$点的度数，是该点连接的所有边的权重之和：

$\sum^{n-1}_{j=0}{w_{ij}}$

则上图的转移概率矩阵为：

In [4]:
P = copy.deepcopy(N)
for i in range(np.shape(N)[0]):
    sum_row = 0.0
    for j in range(np.shape(N)[1]):
        sum_row += N.iloc[i, j]
    for j in range(np.shape(N)[1]):
        P.iloc[i, j] = round(N.iloc[i, j]*100/sum_row)/100
P

,0,1,2,3,4,5,6
0,0.00,0.28,0.00,0.00,0.28,0.00,0.45
1,0.67,0.00,0.16,0.00,0.00,0.16,0.00
2,0.00,0.50,0.00,0.00,0.00,0.50,0.00
3,0.00,0.00,0.00,0.00,0.00,0.00,1.00
4,1.00,0.00,0.00,0.00,0.00,0.00,0.00
5,0.00,0.50,0.50,0.00,0.00,0.00,0.00
6,0.87,0.00,0.00,0.13,0.00,0.00,0.00


In [5]:
P.T

,0,1,2,3,4,5,6
0,0.00,0.67,0.0,0.0,1.0,0.0,0.87
1,0.28,0.00,0.5,0.0,0.0,0.5,0.00
2,0.00,0.16,0.0,0.0,0.0,0.5,0.00
3,0.00,0.00,0.0,0.0,0.0,0.0,0.13
4,0.28,0.00,0.0,0.0,0.0,0.0,0.00
5,0.00,0.16,0.5,0.0,0.0,0.0,0.00
6,0.45,0.00,0.0,1.0,0.0,0.0,0.00


In [6]:
np.dot(P.T.values, [1, 0, 0, 0, 0, 0, 0])

array([0.  , 0.28, 0.  , 0.  , 0.28, 0.  , 0.45])

In [220]:
 v_test = [1, 0, 0, 0, 0, 0, 0]

In [328]:
v_test = 0.85*np.dot(P.T.values, v_test)+[0.15, 0, 0, 0, 0, 0, 0]
v_test

array([0.45327006, 0.14466853, 0.03421725, 0.01832478, 0.11558386,
       0.03421725, 0.16583508])

### 5. 求一个点游走到其他点的概率向量
$p_i$点第一次游走到其他点的概率列向量的计算公式为：

$v_{i,1}=(1-\alpha)P^Tv_{i,0}+\alpha r_i$

其中:
  1. $\alpha$为随机转移概率系数，一般取0.15
  2. $r_i$是一个维的列向量（上图n=7），
$ r_i[k]=\begin{cases}
1 & k = i \\
0 & k \neq i  
\end{cases}$，

即假如$i=0$，$r_i=r_0=
\left[  \begin{array}{l}
1  \\
0  \\
0  \\
0  \\
0  \\
0  \\
0  \end{array} \right]
$
  3. $v_{i,0}$为$p_i$点游走到其他点的初始概率，$v_{i,0}=r_i$
  4. $P^T$表示为矩阵$P$的转置
  5. $v_i$向量的所有元素之和等于1

经过次迭代后，概率会收敛于：
$v_{i,n}=(1-\alpha )P^Tv_{i,n-1}+\alpha r$

即$v_{i,n}=v_{i, n-1}$时或迭代次数达到60次时停止迭代，取最后收敛的$v_{i,n}$为$p_i$点游走到各点的概率，且同样满足上述第5点的条件。

$p_0$点的概率列向量计算结果为

$v_0=\left[ \begin{array}{c}
0.45  \\
0.17  \\
0.04  \\
0.03  \\
0.03  \\
0.04  \\
0.24  \end{array} \right]$

即$p_0$点会留在自己本身的概率为0.45，$p_0$点会游走到$p_1$点的概率为0.17，$\cdots$，$p_0$点会游走到$p_6$点的概率为0.24

In [7]:
demical = 5
matrix_r = np.eye(7)
v_final = []
a = 0.15
list_a = [0.1, 0.3, 0.8, 0.9, 0.9, 0.8, 0.15]
start = time.perf_counter()
Pt = (P.T).values

for j in range(7):
    r0 = np.array(matrix_r[j])
    a = list_a[j]
    v = r0
    for i in range(60):
        v = (1-a)*np.dot(Pt, v)+a*r0
    for i in range(np.size(v)):
        v[i] = round(v[i]*10**demical)/10**demical
    v_final.append(v)
    
v_final_test = []
E = np.identity(np.shape(Pt)[0])
for j in range(7):
    r0 = np.array(matrix_r[j])
    a = list_a[j]

    v = r0
    Pt = (P.T).values
    r0 = np.dot(np.linalg.inv(E-(1-a)*Pt), a*r0)
    for i in range(np.size(r0)):
        r0[i] = round(r0[i]*10**demical)/10**demical
    v_final_test.append(r0)
elapsed = (time.perf_counter() - start)
print("Time used:",elapsed)
v_final = pd.DataFrame(v_final)
v_final = v_final.T
v_final_test = pd.DataFrame(v_final_test)
v_final_test = v_final_test.T
print ("------------------------------")
print ("iteration result is : ")
print(v_final)

print ("------------------------------")
print ("equation result is : ")
print(v_final_test)

Time used: 0.007041300000310002
------------------------------
iteration result is : 
         0        1        2        3        4        5        6
0  0.45475  0.28913  0.01243  0.00791  0.09078  0.01243  0.38757
1  0.14983  0.40559  0.09023  0.00022  0.00255  0.09023  0.11546
2  0.03924  0.06989  0.81129  0.00000  0.00004  0.08402  0.02731
3  0.02409  0.00885  0.00003  0.90118  0.00005  0.00003  0.03637
4  0.11452  0.05667  0.00070  0.00022  0.90254  0.00070  0.09224
5  0.03924  0.06989  0.08402  0.00000  0.00004  0.81129  0.02731
6  0.20571  0.09727  0.00112  0.09047  0.00409  0.00112  0.32917
------------------------------
equation result is : 
         0        1        2        3        4        5        6
0  0.45462  0.28913  0.01243  0.00791  0.09078  0.01243  0.38757
1  0.14988  0.40559  0.09023  0.00022  0.00255  0.09023  0.11545
2  0.03924  0.06989  0.81129  0.00000  0.00004  0.08402  0.02731
3  0.02408  0.00885  0.00003  0.90118  0.00005  0.00003  0.03637
4  0.11457  0.05

In [8]:
result = np.dot([0, 1, 0, 0, 1, 0, 0],v_final.values)
for i in result:
    print(i)

0.26435
0.46226
0.09093000000000001
0.00044
0.9050900000000001
0.09093000000000001
0.2077


In [330]:
for i in range(7):
    print(np.sum(v_final[i]))

0.94864
0.9838300000000001
0.9997499999999999
1.0
0.9998999999999999
0.99975
0.9723599999999999


In [331]:
for i in range(7):
    print(np.sum(v_final_test[i]))

0.94857
0.9838300000000001
0.9997499999999999
1.0
0.9998999999999999
0.99975
0.97235
